#### Note:

Make sure that you have these two scripts running at the same time while running this

ark regsitry start

python sim_node.py

In [76]:
import time
import numpy as np
from arktypes import joint_group_command_t, task_space_command_t, joint_state_t
from arktypes.utils import pack, unpack
from ark.client.comm_infrastructure.instance_node import InstanceNode

ROBOT_NAME = "arkbot" # ArkBot
SIM = False

class HarkControllerNode(InstanceNode):
    def __init__(self):
        super().__init__("arkbot_controller") # ArkBotController
        suffix = "/sim" if SIM else ""
        base = f"{ROBOT_NAME}"

        # pubs
        self.joint_group_command = self.create_publisher(f"{base}/joint_group_command{suffix}", joint_group_command_t)
        self.joint_group_command_real = self.create_publisher(f"{base}/joint_group_command", joint_group_command_t)
        self.task_space_command  = self.create_publisher(f"{base}/cartesian_command{suffix}", task_space_command_t)

        # subs
        self.state = self.create_listener(f"{base}/joint_states{suffix}", joint_state_t)

node = HarkControllerNode()


[WARNING] [23:47:18.873801] - No global configuration provided. Using default system configuration.
[INFO] [23:47:18.875840] - Service: Successfully registered '__DEFAULT_SERVICE/GetInfo/arkbot_controller_42250ee7-0367-453b-aca5-cd4b11b0d72a' with registry.
[INFO] [23:47:18.876873] - Service: Successfully registered '__DEFAULT_SERVICE/SuspendNode/arkbot_controller_42250ee7-0367-453b-aca5-cd4b11b0d72a' with registry.
[INFO] [23:47:18.877795] - Service: Successfully registered '__DEFAULT_SERVICE/RestartNode/arkbot_controller_42250ee7-0367-453b-aca5-cd4b11b0d72a' with registry.
[ERROR] [23:47:18.878308] - Couldn't load config for other 'arkbot_controller'
[OK] [23:47:18.878916] - setup publisher arkbot/joint_group_command[joint_group_command_t]
[OK] [23:47:18.879412] - setup publisher arkbot/joint_group_command[joint_group_command_t]
[OK] [23:47:18.880193] - setup publisher arkbot/cartesian_command[task_space_command_t]
[OK] [23:47:18.880585] - subscribed to arkbot/joint_states[joint_stat

None


---

# Joint Group Control

In [96]:
all_cmd = [
    1.0,   # Revolute 1 (continuous)
     0.0,   # Revolute 2 (rad)
     0.0,   # Revolute 4 (rad)
     0.00,   # Revolute 5 (continuous)
     0.00,   # Revolute 6 (rad)
     0.000   # Slider 7 position in meters (range 0.0 .. 0.0425)
]
node.joint_group_command.publish(pack.joint_group_command(all_cmd, "all"))

In [117]:
home =  [0.0, 0.0, 0.0, 0.00, 0.0, 0.0, 0.0]
approach_upper = [0.4, 0, 1.2, -2.0, 0.0, 2, 0.0]
contact_upper = [0.1, -0.3, 1.3, -2.0, 0.0, 2.2, 0.0]
push_upper = [0.1, -0.3, 1.3, -1.2, 0.0, 1.6, 0.0]
lower_upper = [0.4, -0.3, 1.3, -2.0, 0.0, 2.2, 0.0]
approach_lower = [0.4, -0.3, 1.8, -2.0, 0.0, 2.2, 0.0]
contact_lower = [0.4, -0.4, 1.8, -1.5, 0.0, 2.2, 0.0]
push_lower = [0.4, -0.4, 1.8, -0.5, 0.0, 1.4, 0.0]
retract_lower = [0.4, 0.3, 1.8, -2.0, 0.0, 2.2, 0.0]


In [124]:
node.joint_group_command.publish(pack.joint_group_command(contact_upper, "all"))

In [126]:
node.joint_group_command.publish(pack.joint_group_command(push_upper, "all"))

In [93]:
node.joint_group_command.publish(pack.joint_group_command(home, "all"))

In [95]:
node.joint_group_command.publish(pack.joint_group_command(approach_upper, "all"))

In [163]:
node.joint_group_command.publish(pack.joint_group_command(lower_upper, "all"))

In [164]:
node.joint_group_command.publish(pack.joint_group_command(approach_lower, "all"))

In [168]:
node.joint_group_command.publish(pack.joint_group_command(contact_lower, "all"))

In [169]:
node.joint_group_command.publish(pack.joint_group_command(push_lower, "all"))

In [170]:
node.joint_group_command.publish(pack.joint_group_command(retract_lower, "all"))

In [ ]:
home2 =  [-1.5, 0, 1.2, -2.0, 0.0, 2, 0.0]
first = [-1.2, -0.3, 1.2, -1, 0.0, -0.6, 0.0]
second = [-1.2, -0.3, 1.2, -1.3, 1.7, 0, 0.0]
third = [-1.2, -0.5, 1.30, -1.3, 1.7, 0, 0.0]
fourth = [-1.2, -0.5, 1.31, -1.3, 1.7, 0, 0.2]
fifth = [-1.2, -0.2, 1.2, -1.3, 1.7, 0, 0.2]
sixth = [-1.2, -0.2, 0, -1.3, 1.7, 0, 0.2]
seventh = [-0.2, -0.2, 0, -1.3, 0, 0, 0.2]
eighth2 = [-0.2, -0.5, 1.31, -1.3, 1.7, 0, 0.2]
nineth = [-0.2, -0.3, 1.2, -1, 0.0, -0.6, 0.0]


In [184]:
node.joint_group_command.publish(pack.joint_group_command(eighth2, "all"))

In [128]:
node.joint_group_command.publish(pack.joint_group_command(home2, "all"))

In [147]:
node.joint_group_command.publish(pack.joint_group_command(first, "all"))

In [153]:
node.joint_group_command.publish(pack.joint_group_command(second, "all"))

In [165]:
node.joint_group_command.publish(pack.joint_group_command(third, "all"))

In [167]:
node.joint_group_command.publish(pack.joint_group_command(fourth, "all"))

In [168]:
node.joint_group_command.publish(pack.joint_group_command(fifth, "all"))

In [181]:
node.joint_group_command.publish(pack.joint_group_command(sixth, "all"))

In [182]:
node.joint_group_command.publish(pack.joint_group_command(seventh, "all"))

In [106]:
node.joint_group_command.publish(pack.joint_group_command(eighth2, "all"))

In [107]:
node.joint_group_command.publish(pack.joint_group_command(nineth, "all"))

In [108]:
node.joint_group_command.publish(pack.joint_group_command(home2, "all"))

In [29]:
# save the position offset, click this once when the robot is in the zero position
# unpack.joint_state(node.state.get())

joint_state_data = unpack.joint_state(node.state.get())

joint_positions = joint_state_data[2]

print(joint_positions)



[ 1.56602394e+00  1.36353848e-03  5.06213660e-02  2.06167018e+00
 -5.82912699e-02 -1.39592252e-01 -1.36183406e-01  0.00000000e+00]


In [7]:
# GRIPPER (velocity mode): single value for Slider 7 velocity (m/s)
gripper_vel_open  = [ +0.010 ]   # open slowly
gripper_vel_close = [ -0.010 ]   # close slowly
node.joint_group_command.publish(pack.joint_group_command(gripper_vel_open, "gripper"))

In [8]:
# ARM (position mode): 6 values, Revolute 1..6
arm_cmd = [-0.30, 0.50, 0.30, -1.00, 0.10, 0.80]
node.joint_group_command.publish(pack.joint_group_command(arm_cmd, "arm"))


---

# Task Space Control

In [7]:
xyz_command = np.array([0.3, 0.4, 0.8])
quaternion_command = np.array([1, 0.0, 0.0, 0.0]) # xyz-w
gripper = 1.0 # 0.0 close, 1.0 open

controller.task_space_command.publish(
    pack.task_space_command("all", xyz_command, quaternion_command, gripper))

---


# Joint States

In [287]:
unpack.joint_state(node.state.get())


({'seq': 0, 'stamp': {'sec': 0, 'nsec': 0}, 'frame_id': ''},
 ['Revolute 1',
  'Revolute 2',
  'Revolute 3',
  'Revolute 4',
  'Revolute 5',
  'Revolute 6',
  'Slider 7'],
 array([0.99998503, 0.99998503, 0.        , 0.        , 0.        ,
        0.        , 0.        ]),
 array([0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0.]))

# PYBULLET TESTING

In [37]:
import pybullet as p, pybullet_data as pd, math

p.connect(p.DIRECT)  # use GUI to debug
p.setGravity(0, 0, -9.81)

robot = p.loadURDF("/home/sarthakdas/Ark/ark_bot/arkbot/ark_bot.urdf", [0, 0, 0], useFixedBase=True)

# print("Joints in robot:")
for j in range(p.getNumJoints(robot)):
    print(j, p.getJointInfo(robot, j)[12].decode())

# # Pick correct end effector index after checking printout
ee = 5



0 Link1
1 link2
2 link3
3 link4
4 link5
5 Hand
6 finger1
7 finger2


In [69]:
target_pos = [-0.25, -0.2, 0.27]
target_orn = p.getQuaternionFromEuler([0, math.pi, 0.0])

q = p.calculateInverseKinematics(robot, ee, target_pos, target_orn)

print("IK solution:", q[:p.getNumJoints(robot)])

node.joint_group_command.publish(pack.joint_group_command(q, "all"))
node.joint_group_command_real.publish(pack.joint_group_command(q, "all"))


IK solution: (-0.2912198028348503, -0.3599159317112861, 1.2967921417815351, -1.1937036501084985, -0.26464458133013735, -0.29705273160339246, 0.0, 0.0)


In [35]:
target_pos = [-0.25, -0.2, 0.27]
target_orn = p.getQuaternionFromEuler([0.0, math.pi, 0.0])
target_orn2 = p.getQuaternionFromEuler([0.0, 0.0, 0.0])

q = p.calculateInverseKinematics(robot, ee, target_pos, target_orn)

print("IK solution:", q[:p.getNumJoints(robot)])

node.joint_group_command.publish(pack.joint_group_command(q, "all"))
node.joint_group_command_real.publish(pack.joint_group_command(q, "all"))


IK solution: (-0.2912198028348503, -0.3599159317112861, 1.2967921417815351, -1.1937036501084985, -0.26464458133013735, -0.29705273160339246, 0.0, 0.0)


In [36]:

p.disconnect()

In [55]:
# ---- one-time IK config (put this once, after loading the robot) ----
import math

nJ = p.getNumJoints(robot)
lower, upper, ranges = [], [], []
for j in range(nJ):
    ji = p.getJointInfo(robot, j)
    ll, ul = ji[8], ji[9]
    lower.append(ll)
    upper.append(ul)
    # handle continuous/no-limit joints: fall back to 2*pi range
    jr = (ul - ll) if (ul > ll and abs(ul - ll) < 1e6) else 2*math.pi
    ranges.append(jr)
damping = [0.05] * nJ  # tune if needed

_prev_q = None
def ik_smooth(target_pos, target_orn):
    global _prev_q
    rest = _prev_q if _prev_q is not None else [0.0] * nJ
    q_full = p.calculateInverseKinematics(
        robot, ee, target_pos, target_orn,
        lowerLimits=lower, upperLimits=upper,
        jointRanges=ranges, restPoses=rest,
        jointDamping=damping, residualThreshold=1e-4, maxNumIterations=100
    )
    q = list(q_full[:nJ])
    _prev_q = q  # cache for next call
    return q


In [68]:
import math, time

# =====================
# CONFIG
# =====================
REPEAT_COUNT = 3        # 🔁 Number of figure-eight loops
DURATION_PER_LOOP = 6.0 # seconds per loop
STEPS_PER_LOOP = 200    # number of IK waypoints per loop

# ---- one-time IK setup ----
nJ = p.getNumJoints(robot)
lower, upper, ranges = [], [], []
for j in range(nJ):
    ji = p.getJointInfo(robot, j)
    ll, ul = ji[8], ji[9]
    lower.append(ll)
    upper.append(ul)
    jr = (ul - ll) if (ul > ll and abs(ul - ll) < 1e6) else 2*math.pi
    ranges.append(jr)
damping = [0.05] * nJ  # light damping

_prev_q = None
def ik_smooth(target_pos, target_orn):
    global _prev_q
    rest = _prev_q if _prev_q is not None else [0.0] * nJ
    q_full = p.calculateInverseKinematics(
        robot, ee, target_pos, target_orn,
        lowerLimits=lower, upperLimits=upper,
        jointRanges=ranges, restPoses=rest,
        jointDamping=damping, residualThreshold=1e-4, maxNumIterations=100
    )
    q = list(q_full[:nJ])
    _prev_q = q
    return q

def publish(q):
    node.joint_group_command.publish(pack.joint_group_command(q, "all"))
    # node.joint_group_command_real.publish(pack.joint_group_command(q, "all"))

# ---- figure-eight generator (flat on the floor: constant Z) ----
def draw_figure_eight(center_pos, size_x=0.10, size_y=0.06,
                      euler_orn=(0.0, math.pi, 0.0)):
    cx, cy, cz = center_pos
    quat = p.getQuaternionFromEuler(euler_orn)
    dt = DURATION_PER_LOOP / STEPS_PER_LOOP

    for _ in range(REPEAT_COUNT):          # repeat full loops
        for k in range(STEPS_PER_LOOP + 1):
            t = 2.0 * math.pi * (k / STEPS_PER_LOOP)
            # Gerono lemniscate in X–Y, Z fixed
            x = cx + size_x * math.sin(t)
            y = cy + size_y * math.sin(t) * math.cos(t)
            z = cz  # <-- constant height (floor drawing)
            pos = [x, y, z]
            q = ik_smooth(pos, quat)
            publish(q)
            time.sleep(dt)


# ----------------- run it -----------------
center = [-0.25, -0.2, 0.27]  # set center[2] to your floor height
draw_figure_eight(center_pos=center,
                  size_x=0.10,
                  size_y=0.06,
                  euler_orn=(0.0, math.pi, 0.0))



In [71]:
import math, time

# =====================
# CONFIG
# =====================
REPEAT_COUNT = 3        # 🔁 Number of circles
DURATION_PER_LOOP = 6.0 # seconds per circle
STEPS_PER_LOOP = 200    # number of IK waypoints per circle

# ---- one-time IK setup ----
nJ = p.getNumJoints(robot)
lower, upper, ranges = [], [], []
for j in range(nJ):
    ji = p.getJointInfo(robot, j)
    ll, ul = ji[8], ji[9]
    lower.append(ll)
    upper.append(ul)
    jr = (ul - ll) if (ul > ll and abs(ul - ll) < 1e6) else 2*math.pi
    ranges.append(jr)
damping = [0.05] * nJ  # light damping

_prev_q = None
def ik_smooth(target_pos, target_orn):
    """Inverse kinematics with rest pose seeding for continuity"""
    global _prev_q
    rest = _prev_q if _prev_q is not None else [0.0] * nJ
    q_full = p.calculateInverseKinematics(
        robot, ee, target_pos, target_orn,
        lowerLimits=lower, upperLimits=upper,
        jointRanges=ranges, restPoses=rest,
        jointDamping=damping,
        residualThreshold=1e-4, maxNumIterations=100
    )
    q = list(q_full[:nJ])
    _prev_q = q
    return q

def publish(q):
    node.joint_group_command.publish(pack.joint_group_command(q, "all"))
    node.joint_group_command_real.publish(pack.joint_group_command(q, "all"))

# ---- circle generator (flat on floor: constant Z) ----
def draw_circle(center_pos, radius=0.10, euler_orn=(0.0, math.pi, 0.0)):
    """
    Draw a circle parallel to the floor (X–Y plane) at constant Z.
    """
    cx, cy, cz = center_pos
    quat = p.getQuaternionFromEuler(euler_orn)
    dt = DURATION_PER_LOOP / STEPS_PER_LOOP

    for _ in range(REPEAT_COUNT):  # repeat full loops
        for k in range(STEPS_PER_LOOP + 1):
            theta = 2.0 * math.pi * (k / STEPS_PER_LOOP)
            x = cx + radius * math.cos(theta)
            y = cy + radius * math.sin(theta)
            z = cz  # constant height
            pos = [x, y, z]
            q = ik_smooth(pos, quat)
            publish(q)
            time.sleep(dt)

# ----------------- run it -----------------
center = [-0.25, -0.2, 0.27]  # set Z to just above your floor
draw_circle(center_pos=center,
            radius=0.10,
            euler_orn=(0.0, math.pi, 0.0))


KeyboardInterrupt: 

In [42]:
# --- waypoint 1 ---
target_pos = [-0.2, -0.2, 0.25]
target_orn = p.getQuaternionFromEuler([0, math.pi, 0.0])

q = ik_smooth(target_pos, target_orn)
print("IK solution:", q[:p.getNumJoints(robot)])
node.joint_group_command.publish(pack.joint_group_command(q, "all"))
node.joint_group_command_real.publish(pack.joint_group_command(q, "all"))


IK solution: [-0.1270004947175909, -0.3096205591543132, 1.4263199082610953, -1.4330551546917114, -0.10332860407380073, 0.03852493926037219, 0.0, 0.0]


In [43]:

# --- waypoint 2 ---
target_pos = [-0.25, -0.2, 0.27]
target_orn = p.getQuaternionFromEuler([0.0, math.pi, 0.0])  # (target_orn2 unused here)

q = ik_smooth(target_pos, target_orn)
print("IK solution:", q[:p.getNumJoints(robot)])
node.joint_group_command.publish(pack.joint_group_command(q, "all"))
node.joint_group_command_real.publish(pack.joint_group_command(q, "all"))


IK solution: [-0.09044209546730764, -0.41881192973174103, 1.179681764525028, -1.5390641717846965, -0.062266860537619984, -0.004544190400353901, 0.0, 0.0]


In [ ]:
p.disconnect()